In [9]:
from os import path
import re

fullSub = re.compile(r'<note.+?/note>')
wordSub = re.compile(r'</*w>|<w .*?>')
elemSub = re.compile(r'</*(lb|pb|div|app|pc|hi|supplied).*?>')
gabSub  = re.compile(r'<gap.*/>')
partSub = re.compile(r'<ab.*?part.*?>')
# puntSub = re.compile()

def mss_clean(input_file, output_file):
    out = open(output_file, 'w+')
    with open(input_file) as input_file:
        for line in input_file.readlines():
            line = re.sub(fullSub, '', line)
            line = re.sub(elemSub, '', line)
            line = re.sub(gabSub, ' {gap} ', line)
            line = re.sub(partSub, '', line)
            if line.strip().endswith('</w>'):
                line = line.strip() + ' \n'
            line = re.sub(wordSub, '', line)
            if line.strip().endswith('>'):
                line = line.strip() + '\n'
#             out.write(line.replace('</ab>', '')) # .replace('\n', ''))
            out.write(line.replace('</ab>', '').replace('\n', ''))
#             out.write(line.replace('<w>', '').replace('</w>', ''))
    out.close()
    
mss_clean(path.expanduser(path.join('~/github/pthu/sources/manuscripts/test/20004.xml')), path.expanduser(path.join('~/github/pthu/sources/manuscripts/test/out/20004_clean.xml')))